In [35]:
import pandas as pd

url = '../.../../assets/AmesHousing.csv'
df = pd.read_csv(url) #, engine='pyarrow', dtype_backend='pyarrow')

### Outliers and Z-scores

Goals:

* Calculate the Z-score for the "SalePrice" column using the `calc_z` function.
* Identify outliers based on the Z-score by assigning a boolean column indicating whether the Z-score is greater than or equal to 3 or less than or equal to -3.
* Identify outliers using the IQR (interquartile range) method by assigning a boolean column indicating whether the values are outside the range of median ± 3 * IQR.


In [44]:
# outlier with Z-score
def calc_z(df_, col):
    mean = df_[col].mean()
    std = df_[col].std()
    return (df_[col]-mean)/std

In [45]:
(df
 .pipe(calc_z, col='SalePrice')
)

0       0.428156
1      -0.948795
2      -0.110107
3       0.791170
4       0.113961
          ...   
2925   -0.479380
2926   -0.623334
2927   -0.610816
2928   -0.135142
2929    0.090177
Name: SalePrice, Length: 2930, dtype: float64

In [46]:
(df
 .assign(z_score=calc_z(df, col='SalePrice'))
 .query('z_score.abs() >= 3')
 # .query('z_score <= -3')
)

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice,z_score,z_score_outlier
15,16,527216070,60,RL,47.0,53504,Pave,NaN,IR2,HLS,...,NaN,NaN,0,6,2010,WD,Normal,538000,4.471382,True
44,45,528150070,20,RL,100.0,12919,Pave,NaN,IR1,Lvl,...,NaN,NaN,0,3,2010,New,Partial,611657,5.393401,True
46,47,528176010,20,RL,110.0,14300,Pave,NaN,Reg,HLS,...,NaN,NaN,0,6,2010,WD,Normal,500000,3.995709,True
366,367,527214050,20,RL,63.0,17423,Pave,NaN,IR1,Lvl,...,NaN,NaN,0,7,2009,New,Partial,501837,4.018704,True
421,422,528102140,60,RL,110.0,14257,Pave,NaN,Reg,Lvl,...,NaN,NaN,0,6,2009,WD,Normal,462000,3.520035,True
422,423,528104070,60,RL,104.0,13518,Pave,NaN,Reg,Lvl,...,NaN,NaN,0,7,2009,New,Partial,485000,3.807943,True
423,424,528106020,20,RL,105.0,15431,Pave,NaN,Reg,Lvl,...,NaN,NaN,0,4,2009,WD,Normal,555000,4.684184,True
431,432,528110010,60,RL,97.0,13478,Pave,NaN,IR1,Lvl,...,NaN,NaN,0,6,2009,ConLI,Normal,451950,3.394232,True
432,433,528110020,20,RL,105.0,13693,Pave,NaN,Reg,Lvl,...,NaN,NaN,0,3,2009,WD,Normal,610000,5.372659,True
433,434,528110090,60,RL,107.0,13891,Pave,NaN,Reg,Lvl,...,NaN,NaN,0,1,2009,New,Partial,582933,5.033841,True


In [47]:
df = df.assign(
    z_score=calc_z(df, col='SalePrice'),
    z_score_outlier=lambda d: d['z_score'].abs() > 3
)


In [48]:
def calc_iqr_outlier(df, col):
    ser = df[col]
    iqr = ser.quantile(.75) - ser.quantile(.25)
    med = ser.median()
    small_mask = ser < med-iqr*3
    large_mask = ser > med+iqr*3
    return small_mask | large_mask

In [41]:
# df[
# calc_iqr_outlier(df, 'SalePrice')
# ]

In [54]:
df = (df
 .assign(iqr_outlier=calc_iqr_outlier(df, col='SalePrice'))
 #.query('iqr_outlier')
)

In [55]:
df.columns

Index(['Order', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt',
      

In [56]:
df

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice,z_score,z_score_outlier,iqr_outlier
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,NaN,0,5,2010,WD,Normal,215000,0.428156,False,False
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,NaN,0,6,2010,WD,Normal,105000,-0.948795,False,False
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,Gar2,12500,6,2010,WD,Normal,172000,-0.110107,False,False
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,NaN,0,4,2010,WD,Normal,244000,0.791170,False,False
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,NaN,0,3,2010,WD,Normal,189900,0.113961,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,923275080,80,RL,37.0,7937,Pave,NaN,IR1,Lvl,...,NaN,0,3,2006,WD,Normal,142500,-0.479380,False,False
2926,2927,923276100,20,RL,NaN,8885,Pave,NaN,IR1,Low,...,NaN,0,6,2006,WD,Normal,131000,-0.623334,False,False
2927,2928,923400125,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,...,Shed,700,7,2006,WD,Normal,132000,-0.610816,False,False
2928,2929,924100070,20,RL,77.0,10010,Pave,NaN,Reg,Lvl,...,NaN,0,4,2006,WD,Normal,170000,-0.135142,False,False
